In [ ]:
%pip install requests

In [ ]:
from selenium import webdriver
from datetime import timedelta, datetime
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.common.action_chains import ActionChains
import re 
import requests

In [ ]:
driver = webdriver.Chrome()
driver.get('https://ria.ru/economy')
data = []
last_i = -1

while True:
    html = driver.page_source
    soup = BeautifulSoup(html)

    containers = soup.find_all("div", attrs={"class": "list-item", "data-type": "article"})

    for i, container in enumerate(containers):

        if i < last_i:
            continue
        
        header = container.find('a', attrs= {'class': re.compile('^list-item__title\D*$')})
        header_text = header.text
        href = header['href']

        # open the page
        page = requests.get(href)
        soup_inner = BeautifulSoup(page.text, 'html.parser')
        dttime_el = soup_inner.find('div', attrs={'class': 'article__info-date'}).find('a')
        dttime = dttime_el.text if dttime_el is not None else "None"
        #author = "NOT FOUND"
        brief_el = soup_inner.find('h1', attrs={'class': 'article__second-title'})
        brief = "None" if brief_el is None else brief_el.text
        tags_element = soup_inner.find('div', attrs={'class': 'article__tags'}).find_all('a')
        tags = ",".join([el.text for el in tags_element])


        #//*[@id="content"]/div/div[1]/div/div[3]

        print(i, header_text, dttime, href, brief, tags)
        data.append((header_text, dttime, href, brief, tags)) 

        last_i = i

        #data.append((header, timedata, ref))
        #print(f"Found header: {header}, date: {timedata}, ref: {ref}")

    #//*[@id="content"]/div/div[1]/div/div[3]
    bottom = driver.find_element(By.XPATH, '//*[@id="content"]/div/div[1]/div/div[3]')
    location = bottom.location
    size = bottom.size
    action = ActionChains(driver) 
    action.move_to_element(bottom).perform()
    driver.execute_script("arguments[0].click();", bottom)
    time.sleep(1)


In [ ]:
df = pd.DataFrame(data, columns = ['header', 'date', 'link', 'brief', 'tags'])
df = df.drop_duplicates()
df.head()

In [ ]:
df.to_csv("../data/ria_temp.csv", index=False)